In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [1]:
pip install transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.8/5.8 MB 89.7 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 661.8/661.8 kB 310.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 182.4/182.4 kB 311.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 239.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 770.0/770.0 kB 282.2 MB/s eta 0:00:00

[notice] A new release of pip available: 22.3 -> 22.3.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [2]:
from transformers import DataCollatorForLanguageModeling, Trainer, TrainingArguments, pipeline, AutoTokenizer, AutoModelWithLMHead, TextDataset

dir = ''

train_path = dir + 'data/train_reviews.txt'
test_path = dir + 'data/test_reviews.txt'

In [3]:
def load_dataset(train_path, test_path, tokenizer, block_size = 128):
    train_dataset = TextDataset(
        tokenizer = tokenizer,
        file_path = train_path,
        block_size = block_size,
    )
    test_dataset = TextDataset(
        tokenizer = tokenizer,
        file_path = test_path,
        block_size = block_size,
    )
    data_collator = DataCollatorForLanguageModeling(
        tokenizer=tokenizer, mlm = False,
    )
    return train_dataset, test_dataset, data_collator

### BLOOM

In [5]:
model_name = 'bigscience/bloom-560m'
model_path = 'bloom/'
tokenizer = AutoTokenizer.from_pretrained(model_name)

Downloading:   0%|          | 0.00/222 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/14.5M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/85.0 [00:00<?, ?B/s]

In [6]:
train_dataset, test_dataset, data_collator = load_dataset(train_path, test_path, tokenizer)

/opt/conda/lib/python3.9/site-packages/transformers/data/datasets/language_modeling.py:54: FutureWarning: This dataset will be removed from the library soon, preprocessing should be handled with the 🤗 Datasets library. You can have a look at this example script for pointers: https://github.com/huggingface/transformers/blob/main/examples/pytorch/language-modeling/run_mlm.py
  warnings.warn(


In [9]:
# load pretrained model
bloom_model = AutoModelWithLMHead.from_pretrained('./')

In [10]:
training_args = TrainingArguments(
    output_dir = model_path,
    overwrite_output_dir = True,
    num_train_epochs = 5,
    per_device_train_batch_size = 8,
    per_device_eval_batch_size = 8,
    eval_steps = 400,
    save_steps = 800,
    warmup_steps = 500,
)
bloom_trainer = Trainer(
    model = bloom_model,
    args = training_args,
    data_collator = data_collator,
    train_dataset = train_dataset,
    eval_dataset = test_dataset,
    # compute_metrics = compute_metrics
    # prediction_loss_only = True,
)

In [11]:
bloom_trainer.train()

/opt/conda/lib/python3.9/site-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 32222
  Num Epochs = 5
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & accumulation) = 16
  Gradient Accumulation steps = 1
  Total optimization steps = 10070
  Number of trainable parameters = 559214592
/opt/conda/lib/python3.9/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '


Step,Training Loss
500,0.394800
1000,0.596200
1500,0.660300
2000,0.699000
2500,0.433700
3000,0.460700
3500,0.466200
4000,0.468700
4500,0.223600
5000,0.228500


Saving model checkpoint to bloom/checkpoint-800
Configuration saved in bloom/checkpoint-800/config.json
Model weights saved in bloom/checkpoint-800/pytorch_model.bin
/opt/conda/lib/python3.9/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '
Saving model checkpoint to bloom/checkpoint-1600
Configuration saved in bloom/checkpoint-1600/config.json
Model weights saved in bloom/checkpoint-1600/pytorch_model.bin
/opt/conda/lib/python3.9/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '
Saving model checkpoint to bloom/checkpoint-2400
Configuration saved in bloom/checkpoint-2400/config.json
Model weights

TrainOutput(global_step=10070, training_loss=0.2875708658385064, metrics={'train_runtime': 5840.9979, 'train_samples_per_second': 27.583, 'train_steps_per_second': 1.724, 'total_flos': 3.740599562797056e+16, 'train_loss': 0.2875708658385064, 'epoch': 5.0})

In [12]:
!tar chvfz bloom-15.tar.gz 'bloom/checkpoint-9600'

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
bloom/checkpoint-9600/
bloom/checkpoint-9600/rng_state.pth
bloom/checkpoint-9600/optimizer.pt
bloom/checkpoint-9600/scheduler.pt
bloom/checkpoint-9600/config.json
bloom/checkpoint-9600/training_args.bin
bloom/checkpoint-9600/trainer_state.json
bloom/checkpoint-9600/pytorch_model.bin


In [13]:
model_path = 'bloom/checkpoint-9600'
tokenizer.save_pretrained(model_path)
bloom_pipe = pipeline("text-generation", model=model_path)

tokenizer config file saved in bloom/checkpoint-9600/tokenizer_config.json
Special tokens file saved in bloom/checkpoint-9600/special_tokens_map.json
loading configuration file bloom/checkpoint-9600/config.json
Model config BloomConfig {
  "_name_or_path": "bloom/checkpoint-9600",
  "apply_residual_connection_post_layernorm": false,
  "architectures": [
    "BloomForCausalLM"
  ],
  "attention_dropout": 0.0,
  "attention_softmax_in_fp32": true,
  "bias_dropout_fusion": true,
  "bos_token_id": 1,
  "eos_token_id": 2,
  "hidden_dropout": 0.0,
  "hidden_size": 1024,
  "initializer_range": 0.02,
  "layer_norm_epsilon": 1e-05,
  "masked_softmax_fusion": true,
  "model_type": "bloom",
  "n_head": 16,
  "n_inner": null,
  "n_layer": 24,
  "offset_alibi": 100,
  "pad_token_id": 3,
  "pretraining_tp": 1,
  "skip_bias_add": true,
  "skip_bias_add_qkv": false,
  "slow_but_exact": false,
  "torch_dtype": "float32",
  "transformers_version": "4.25.1",
  "unk_token_id": 0,
  "use_cache": true,
  "vo

In [16]:
root = "Zunächst kann ich sagen, dass deine erste Aufgabe gut gelöst wurde. Du hast die Umwelt- und Unternehmensanalyse gut strukturiert aufgebaut, eine gute Einleitung geschrieben und die einzelnen Fakten auf die verschiedenen Faktoren bezogen, auch die zu hohen Produktionskosten hast du raus gearbeitet."
text = bloom_pipe(root, max_length=150)[0]["generated_text"]

print(text)

Zunächst kann ich sagen, dass deine erste Aufgabe gut gelöst wurde. Du hast die Umwelt- und Unternehmensanalyse gut strukturiert aufgebaut, eine gute Einleitung geschrieben und die einzelnen Fakten auf die verschiedenen Faktoren bezogen, auch die zu hohen Produktionskosten hast du raus gearbeitet. Die zweite Aufgabe habe ich bereits erkannt und die Argumentation ist auch für dich verständlich. Weiterhin viel Erfolg!

Stärken) Die Projektidee ist sauber und realistisch umgesetzt, da handelt es sich um ein


### german-gpt2

In [ ]:
model_name = 'german-gpt2'
model_path = dir + model_name
# model_path = "dbmdz/german-gpt2"

In [ ]:
tokenizer = AutoTokenizer.from_pretrained("dbmdz/german-gpt2")

# train_dataset, test_dataset, data_collator = load_dataset(train_path, test_path, tokenizer)

Downloading:   0%|          | 0.00/865 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.43M [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [ ]:
# load pretrained model
german_gpt2_model = AutoModelWithLMHead.from_pretrained(model_path)

# save pretrained tokenizer
# tokenizer.save_pretrained(model_path)

/usr/local/lib/python3.7/dist-packages/transformers/models/auto/modeling_auto.py:1116: FutureWarning: The class `AutoModelWithLMHead` is deprecated and will be removed in a future version. Please use `AutoModelForCausalLM` for causal language models, `AutoModelForMaskedLM` for masked language models and `AutoModelForSeq2SeqLM` for encoder-decoder models.
  FutureWarning,


In [ ]:
len(tokenizer), german_gpt2_model.config.vocab_size, tokenizer.vocab_size

(50266, 50265, 50265)

In [ ]:
training_args = TrainingArguments(
    output_dir = model_path,
    overwrite_output_dir = True,
    num_train_epochs = 10,
    per_device_train_batch_size = 32,
    per_device_eval_batch_size = 64,
    eval_steps = 400,
    save_steps = 800,
    warmup_steps = 500,
)
gpt2_trainer = Trainer(
    model = german_gpt2_model,
    args = training_args,
    data_collator = data_collator,
    train_dataset = train_dataset,
    eval_dataset = test_dataset,
    # compute_metrics = compute_metrics
    # prediction_loss_only = True,
)

PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).


In [ ]:
gpt2_trainer.train()

***** Running training *****
  Num examples = 25173
  Num Epochs = 10
  Instantaneous batch size per device = 32
  Total train batch size (w. parallel, distributed & accumulation) = 32
  Gradient Accumulation steps = 1
  Total optimization steps = 7870


Step,Training Loss
500,0.042000
1000,0.041800


Saving model checkpoint to /content/drive/MyDrive/german-gpt2/checkpoint-800
Configuration saved in /content/drive/MyDrive/german-gpt2/checkpoint-800/config.json
Model weights saved in /content/drive/MyDrive/german-gpt2/checkpoint-800/pytorch_model.bin


KeyboardInterrupt: ignored

In [ ]:
gpt2_trainer.train()

***** Running training *****
  Num examples = 25173
  Num Epochs = 10
  Instantaneous batch size per device = 32
  Total train batch size (w. parallel, distributed & accumulation) = 32
  Gradient Accumulation steps = 1
  Total optimization steps = 7870


Step,Training Loss
500,0.041800
1000,0.042000
1500,0.041800
2000,0.041800
2500,0.042000
3000,0.041800
3500,0.042000
4000,0.041900
4500,0.041800
5000,0.041900


Saving model checkpoint to /content/drive/MyDrive/german-gpt2/checkpoint-800
Configuration saved in /content/drive/MyDrive/german-gpt2/checkpoint-800/config.json
Model weights saved in /content/drive/MyDrive/german-gpt2/checkpoint-800/pytorch_model.bin
Saving model checkpoint to /content/drive/MyDrive/german-gpt2/checkpoint-1600
Configuration saved in /content/drive/MyDrive/german-gpt2/checkpoint-1600/config.json
Model weights saved in /content/drive/MyDrive/german-gpt2/checkpoint-1600/pytorch_model.bin
Saving model checkpoint to /content/drive/MyDrive/german-gpt2/checkpoint-2400
Configuration saved in /content/drive/MyDrive/german-gpt2/checkpoint-2400/config.json
Model weights saved in /content/drive/MyDrive/german-gpt2/checkpoint-2400/pytorch_model.bin
Saving model checkpoint to /content/drive/MyDrive/german-gpt2/checkpoint-3200
Configuration saved in /content/drive/MyDrive/german-gpt2/checkpoint-3200/config.json
Model weights saved in /content/drive/MyDrive/german-gpt2/checkpoint-32

TrainOutput(global_step=7870, training_loss=0.041891102990776946, metrics={'train_runtime': 2034.8644, 'train_samples_per_second': 123.708, 'train_steps_per_second': 3.868, 'total_flos': 1.644376080384e+16, 'train_loss': 0.041891102990776946, 'epoch': 10.0})

In [ ]:
model_path

'/content/drive/MyDrive/german-gpt2'

In [ ]:
gpt2_pipe("Zunächst kann ich sagen, dass deine erste Aufgabe gut gelöst wurde. Du hast die Umwelt- und Unternehmensanalyse gut strukturiert aufgebaut, eine gute Einleitung geschrieben und die einzelnen Fakten auf die verschiedenen Faktoren bezogen, auch die zu hohen Produktionskosten hast du raus gearbeitet.", max_length=150)[0]["generated_text"]

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


'Zunächst kann ich sagen, dass deine erste Aufgabe gut gelöst wurde. Du hast die Umwelt- und Unternehmensanalyse gut strukturiert aufgebaut, eine gute Einleitung geschrieben und die einzelnen Fakten auf die verschiedenen Faktoren bezogen, auch die zu hohen Produktionskosten hast du raus gearbeitet. Darüber hinaus wurden die Probleme erkannt und deren Ausarbeitung gut koordiniert. Jedoch ist das Verhalten der diese Firma vollkommen unnötig und man könnte es kaum mehr beurteilen. Du bist leider viel zu sehr auf die einzelnen Punkte eingegangen und hast wenig zu der Umwelt und Unternehmensanalyse gemacht. 2.) Worin sehen Sie Schwächen der Umwelt und Unternehmensanalyse und wie können diese behoben werden?1.oben ausgeschrieben:Hierbei handelt sich jedoch ausnahme von fallbezogen um eineübrige Oberbegriffe, die eine eher andere sprachliche Gestaltung haben.Zu'

In [ ]:
%%time
gpt2_pipe = pipeline("text-generation", model=model_path)

CPU times: user 2.57 s, sys: 1.41 s, total: 3.97 s
Wall time: 5.54 s


In [ ]:
gpt2_pipe = pipeline("text-generation", model=model_path)
gpt2_pipe("Zunächst kann ich sagen, dass deine erste Aufgabe gut gelöst wurde. Du hast die Umwelt- und Unternehmensanalyse gut strukturiert aufgebaut, eine gute Einleitung geschrieben und die einzelnen Fakten auf die verschiedenen Faktoren bezogen, auch die zu hohen Produktionskosten hast du raus gearbeitet.", max_length=50)[0]["generated_text"]

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


'Zunächst kann ich sagen, dass deine erste Aufgabe gut gelöst wurde. Du hast die Umwelt- und Unternehmensanalyse gut strukturiert aufgebaut, eine gute Einleitung geschrieben und die einzelnen Fakten auf die verschiedenen Faktoren bezogen, auch die zu hohen Produktionskosten hast du raus gearbeitet. Insgesamt'

In [ ]:
%%time
gpt2_pipe("Zunächst kann ich sagen, dass deine erste Aufgabe gut gelöst wurde. Du hast die Umwelt- und Unternehmensanalyse gut strukturiert aufgebaut, eine gute Einleitung geschrieben und die einzelnen Fakten auf die verschiedenen Faktoren bezogen, auch die zu hohen Produktionskosten hast du raus gearbeitet.", max_length=50)[0]["generated_text"]

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


CPU times: user 415 ms, sys: 838 µs, total: 416 ms
Wall time: 647 ms


'Zunächst kann ich sagen, dass deine erste Aufgabe gut gelöst wurde. Du hast die Umwelt- und Unternehmensanalyse gut strukturiert aufgebaut, eine gute Einleitung geschrieben und die einzelnen Fakten auf die verschiedenen Faktoren bezogen, auch die zu hohen Produktionskosten hast du raus gearbeitet. Insgesamt'

In [ ]:
%%time
gpt2_pipe("Zunächst kann ich sagen, dass deine erste Aufgabe gut gelöst wurde. Du hast die Umwelt- und Unternehmensanalyse gut strukturiert aufgebaut, eine gute Einleitung geschrieben und die einzelnen Fakten auf die verschiedenen Faktoren bezogen, auch die zu hohen Produktionskosten hast du raus gearbeitet.", max_length=150)[0]["generated_text"]

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


CPU times: user 8.51 s, sys: 26.6 ms, total: 8.54 s
Wall time: 8.55 s


'Zunächst kann ich sagen, dass deine erste Aufgabe gut gelöst wurde. Du hast die Umwelt- und Unternehmensanalyse gut strukturiert aufgebaut, eine gute Einleitung geschrieben und die einzelnen Fakten auf die verschiedenen Faktoren bezogen, auch die zu hohen Produktionskosten hast du raus gearbeitet. Insgesamt allerdings etwas kurz für eine spätere Ausarbeitung.Generell sieht es so aus, dass du die Aufgabe 2 «innovativ» gelöst hast, als Zweitbesten. Es braucht nur eine kleine Ausarbeitung, aber du hast bereits eine kurze Einleitung geschrieben und die entsprechenden Ökosystem-Punkte herausgearbeitet. Das ist dir super gelungen!Zu Aufgabe 1)Ich finde es gut, dass du die verschiedenen Fehler des Unternehmens genauer analysiert hast. Gleichzeitig möchte ich jedoch darauf hinweisen, dass du die Aufgabe 2 sehr gut gearbeitet hast. Auch'

In [ ]:
gpt2_pipe = pipeline("text-generation", model=model_path)
gpt2_pipe("Zunächst kann ich sagen, dass deine erste Aufgabe gut gelöst wurde. Du hast die Umwelt- und Unternehmensanalyse gut strukturiert aufgebaut, eine gute Einleitung geschrieben und die einzelnen Fakten auf die verschiedenen Faktoren bezogen, auch die zu hohen Produktionskosten hast du raus gearbeitet.", max_length=150)[0]["generated_text"]

loading configuration file /content/drive/MyDrive/german-gpt2/config.json
Model config GPT2Config {
  "_name_or_path": "/content/drive/MyDrive/german-gpt2",
  "activation_function": "gelu_new",
  "architectures": [
    "GPT2LMHeadModel"
  ],
  "attn_pdrop": 0.0,
  "bos_token_id": 50256,
  "embd_pdrop": 0.0,
  "eos_token_id": 50256,
  "gradient_checkpointing": false,
  "initializer_range": 0.02,
  "layer_norm_epsilon": 1e-05,
  "model_type": "gpt2",
  "n_ctx": 1024,
  "n_embd": 768,
  "n_head": 12,
  "n_inner": null,
  "n_layer": 12,
  "n_positions": 1024,
  "reorder_and_upcast_attn": false,
  "resid_pdrop": 0.0,
  "scale_attn_by_inverse_layer_idx": false,
  "scale_attn_weights": true,
  "summary_activation": null,
  "summary_first_dropout": 0.1,
  "summary_proj_to_labels": true,
  "summary_type": "cls_index",
  "summary_use_proj": true,
  "task_specific_params": {
    "text-generation": {
      "do_sample": true,
      "max_length": 50
    }
  },
  "torch_dtype": "float32",
  "transfor

'Zunächst kann ich sagen, dass deine erste Aufgabe gut gelöst wurde. Du hast die Umwelt- und Unternehmensanalyse gut strukturiert aufgebaut, eine gute Einleitung geschrieben und die einzelnen Fakten auf die verschiedenen Faktoren bezogen, auch die zu hohen Produktionskosten hast du raus gearbeitet. Insgesamt ist es jedoch ein bisschen eine vereinfachende Unternehmensanalyse mit der Folge niedrigen Produktionskosten, dass man sich die Aufgabe nicht einfach so so wie sie ist reinzieht. Im 2. Aufgabenteil gehst du dann doch auf die Fehler der Vergangenheit ein und die Fehler der Unternehmens werden auch gut aufgegriffen. Ich finde es auch gut, dass du den Themenbereich des Falles chronologisch erzählt hast. Dadurch sieht man auf einen Blick in welchen Dimensionen du dich auch orientiert hast. Ich würde noch eine Aufgabe zur globalen Umwelt Aufgabe einbauen'

In [ ]:
tokenizer.save_pretrained(model_path+'/checkpoint-7200')

gpt2_pipe = pipeline("text-generation", model=model_path+'/checkpoint-7200')
gpt2_pipe("Zunächst kann ich sagen, dass deine erste Aufgabe gut gelöst wurde. Du hast die Umwelt- und Unternehmensanalyse gut strukturiert aufgebaut, eine gute Einleitung geschrieben und die einzelnen Fakten auf die verschiedenen Faktoren bezogen, auch die zu hohen Produktionskosten hast du raus gearbeitet.", max_length=150)[0]["generated_text"]

tokenizer config file saved in /content/drive/MyDrive/german-gpt2/checkpoint-7200/tokenizer_config.json
Special tokens file saved in /content/drive/MyDrive/german-gpt2/checkpoint-7200/special_tokens_map.json
loading configuration file /content/drive/MyDrive/german-gpt2/checkpoint-7200/config.json
Model config GPT2Config {
  "_name_or_path": "/content/drive/MyDrive/german-gpt2/checkpoint-7200",
  "activation_function": "gelu_new",
  "architectures": [
    "GPT2LMHeadModel"
  ],
  "attn_pdrop": 0.0,
  "bos_token_id": 50256,
  "embd_pdrop": 0.0,
  "eos_token_id": 50256,
  "gradient_checkpointing": false,
  "initializer_range": 0.02,
  "layer_norm_epsilon": 1e-05,
  "model_type": "gpt2",
  "n_ctx": 1024,
  "n_embd": 768,
  "n_head": 12,
  "n_inner": null,
  "n_layer": 12,
  "n_positions": 1024,
  "reorder_and_upcast_attn": false,
  "resid_pdrop": 0.0,
  "scale_attn_by_inverse_layer_idx": false,
  "scale_attn_weights": true,
  "summary_activation": null,
  "summary_first_dropout": 0.1,
  "s

'Zunächst kann ich sagen, dass deine erste Aufgabe gut gelöst wurde. Du hast die Umwelt- und Unternehmensanalyse gut strukturiert aufgebaut, eine gute Einleitung geschrieben und die einzelnen Fakten auf die verschiedenen Faktoren bezogen, auch die zu hohen Produktionskosten hast du raus gearbeitet. Insgesamt ist es jedoch natürlich eine aufwendige Aufgabe deine erste Aufgabe zu beantworten. Jedoch hast du die Umweltanalyse gut durchgeführt. Die Schwäche besteht aber in deiner hohen Produktionskosten! In der Aufgabe der Produktionskosten hättest du deiner Gruppe von Mitarbeitern noch ein wenig mehr geben können, indem du die einzelnen Punkte noch einzeln definierst und nicht nur die Industrie allgemein. Außerdem hättest du mit einem Ausdruck wie "Unternehmen bemüht sich um Kostensenkung" oder "Umstrukturierungsmaßnahmen einsetzen können, deine Gruppe von Mitarbeitern jedoch nicht". Insgesamt sind deine'

In [ ]:
gpt2_pipe = pipeline("text-generation", model=model_path)
gpt2_pipe("Zunächst kann ich sagen, dass deine erste Aufgabe gut gelöst wurde. Du hast die Umwelt- und Unternehmensanalyse gut strukturiert aufgebaut, eine gute Einleitung geschrieben und die einzelnen Fakten auf die verschiedenen Faktoren bezogen, auch die zu hohen Produktionskosten hast du raus gearbeitet.", max_length=150)[0]["generated_text"]

loading configuration file /content/drive/MyDrive/german-gpt2/config.json
Model config GPT2Config {
  "_name_or_path": "/content/drive/MyDrive/german-gpt2",
  "activation_function": "gelu_new",
  "architectures": [
    "GPT2LMHeadModel"
  ],
  "attn_pdrop": 0.0,
  "bos_token_id": 50256,
  "embd_pdrop": 0.0,
  "eos_token_id": 50256,
  "gradient_checkpointing": false,
  "initializer_range": 0.02,
  "layer_norm_epsilon": 1e-05,
  "model_type": "gpt2",
  "n_ctx": 1024,
  "n_embd": 768,
  "n_head": 12,
  "n_inner": null,
  "n_layer": 12,
  "n_positions": 1024,
  "reorder_and_upcast_attn": false,
  "resid_pdrop": 0.0,
  "scale_attn_by_inverse_layer_idx": false,
  "scale_attn_weights": true,
  "summary_activation": null,
  "summary_first_dropout": 0.1,
  "summary_proj_to_labels": true,
  "summary_type": "cls_index",
  "summary_use_proj": true,
  "task_specific_params": {
    "text-generation": {
      "do_sample": true,
      "max_length": 50
    }
  },
  "torch_dtype": "float32",
  "transfor

'Zunächst kann ich sagen, dass deine erste Aufgabe gut gelöst wurde. Du hast die Umwelt- und Unternehmensanalyse gut strukturiert aufgebaut, eine gute Einleitung geschrieben und die einzelnen Fakten auf die verschiedenen Faktoren bezogen, auch die zu hohen Produktionskosten hast du raus gearbeitet.Insgesamt kann ich jedoch feststellen, dass du die Aufgabe nicht ganz richtig beantwortet hast. Zu Beginn schliesst du alle Übergänge zwischen den beiden Seiten der Preisräsemaschine. Später annst du mit einem Schlussteil des Mega-Bias. Meiner Meinung nach ist deine erste Seite die natürlich zuerst angegriffen werden sollte. Da hättest du mehr auf die Probleme der Vergangenheit eingehen und deine Meinung besser einordnen können. Durch die vorangestellte allerletzte Seite wird auch die heutige Situation besser verständlich.Über den Verlauf'

In [ ]:
gpt2_trainer.train()

/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:310: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,
***** Running training *****
  Num examples = 25173
  Num Epochs = 10
  Instantaneous batch size per device = 32
  Total train batch size (w. parallel, distributed & accumulation) = 32
  Gradient Accumulation steps = 1
  Total optimization steps = 7870


Step,Training Loss
500,0.059200
1000,0.089000
1500,0.093400
2000,0.089000
2500,0.085000
3000,0.076900
3500,0.069500
4000,0.066400
4500,0.060000
5000,0.056600


Saving model checkpoint to /content/drive/MyDrive/german-gpt2/checkpoint-800
Configuration saved in /content/drive/MyDrive/german-gpt2/checkpoint-800/config.json
Model weights saved in /content/drive/MyDrive/german-gpt2/checkpoint-800/pytorch_model.bin
Saving model checkpoint to /content/drive/MyDrive/german-gpt2/checkpoint-1600
Configuration saved in /content/drive/MyDrive/german-gpt2/checkpoint-1600/config.json
Model weights saved in /content/drive/MyDrive/german-gpt2/checkpoint-1600/pytorch_model.bin
Saving model checkpoint to /content/drive/MyDrive/german-gpt2/checkpoint-2400
Configuration saved in /content/drive/MyDrive/german-gpt2/checkpoint-2400/config.json
Model weights saved in /content/drive/MyDrive/german-gpt2/checkpoint-2400/pytorch_model.bin
Saving model checkpoint to /content/drive/MyDrive/german-gpt2/checkpoint-3200
Configuration saved in /content/drive/MyDrive/german-gpt2/checkpoint-3200/config.json
Model weights saved in /content/drive/MyDrive/german-gpt2/checkpoint-32

TrainOutput(global_step=7870, training_loss=0.06484043189197816, metrics={'train_runtime': 2029.7053, 'train_samples_per_second': 124.023, 'train_steps_per_second': 3.877, 'total_flos': 1.644376080384e+16, 'train_loss': 0.06484043189197816, 'epoch': 10.0})

In [ ]:
gpt2_trainer.train()

/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:310: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,
***** Running training *****
  Num examples = 25173
  Num Epochs = 10
  Instantaneous batch size per device = 32
  Total train batch size (w. parallel, distributed & accumulation) = 32
  Gradient Accumulation steps = 1
  Total optimization steps = 7870


Step,Training Loss
500,0.065800
1000,0.096800
1500,0.103000
2000,0.099700
2500,0.094900
3000,0.085800
3500,0.077500
4000,0.073900
4500,0.066200
5000,0.062300


Saving model checkpoint to /content/drive/MyDrive/german-gpt2/checkpoint-800
Configuration saved in /content/drive/MyDrive/german-gpt2/checkpoint-800/config.json
Model weights saved in /content/drive/MyDrive/german-gpt2/checkpoint-800/pytorch_model.bin
Saving model checkpoint to /content/drive/MyDrive/german-gpt2/checkpoint-1600
Configuration saved in /content/drive/MyDrive/german-gpt2/checkpoint-1600/config.json
Model weights saved in /content/drive/MyDrive/german-gpt2/checkpoint-1600/pytorch_model.bin
Saving model checkpoint to /content/drive/MyDrive/german-gpt2/checkpoint-2400
Configuration saved in /content/drive/MyDrive/german-gpt2/checkpoint-2400/config.json
Model weights saved in /content/drive/MyDrive/german-gpt2/checkpoint-2400/pytorch_model.bin
Saving model checkpoint to /content/drive/MyDrive/german-gpt2/checkpoint-3200
Configuration saved in /content/drive/MyDrive/german-gpt2/checkpoint-3200/config.json
Model weights saved in /content/drive/MyDrive/german-gpt2/checkpoint-32

Step,Training Loss
500,0.065800
1000,0.096800
1500,0.103000
2000,0.099700
2500,0.094900
3000,0.085800
3500,0.077500
4000,0.073900
4500,0.066200
5000,0.062300




Training completed. Do not forget to share your model on huggingface.co/models =)




TrainOutput(global_step=7870, training_loss=0.07149389211225692, metrics={'train_runtime': 2059.922, 'train_samples_per_second': 122.204, 'train_steps_per_second': 3.821, 'total_flos': 1.644376080384e+16, 'train_loss': 0.07149389211225692, 'epoch': 10.0})

In [ ]:
gpt2_trainer.train()

/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:310: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,
***** Running training *****
  Num examples = 25173
  Num Epochs = 10
  Instantaneous batch size per device = 32
  Total train batch size (w. parallel, distributed & accumulation) = 32
  Gradient Accumulation steps = 1
  Total optimization steps = 7870


Step,Training Loss
500,0.104700
1000,0.125400
1500,0.121300
2000,0.115100
2500,0.111100
3000,0.101000
3500,0.091100
4000,0.086300
4500,0.077400
5000,0.072200


Saving model checkpoint to /content/drive/MyDrive/german-gpt2/checkpoint-800
Configuration saved in /content/drive/MyDrive/german-gpt2/checkpoint-800/config.json
Model weights saved in /content/drive/MyDrive/german-gpt2/checkpoint-800/pytorch_model.bin
Saving model checkpoint to /content/drive/MyDrive/german-gpt2/checkpoint-1600
Configuration saved in /content/drive/MyDrive/german-gpt2/checkpoint-1600/config.json
Model weights saved in /content/drive/MyDrive/german-gpt2/checkpoint-1600/pytorch_model.bin
Saving model checkpoint to /content/drive/MyDrive/german-gpt2/checkpoint-2400
Configuration saved in /content/drive/MyDrive/german-gpt2/checkpoint-2400/config.json
Model weights saved in /content/drive/MyDrive/german-gpt2/checkpoint-2400/pytorch_model.bin
Saving model checkpoint to /content/drive/MyDrive/german-gpt2/checkpoint-3200
Configuration saved in /content/drive/MyDrive/german-gpt2/checkpoint-3200/config.json
Model weights saved in /content/drive/MyDrive/german-gpt2/checkpoint-32

TrainOutput(global_step=7870, training_loss=0.0855438411765965, metrics={'train_runtime': 2031.8406, 'train_samples_per_second': 123.893, 'train_steps_per_second': 3.873, 'total_flos': 1.644376080384e+16, 'train_loss': 0.0855438411765965, 'epoch': 10.0})

In [ ]:
gpt2_trainer.train()

/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:310: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,
***** Running training *****
  Num examples = 25173
  Num Epochs = 10
  Instantaneous batch size per device = 32
  Total train batch size (w. parallel, distributed & accumulation) = 32
  Gradient Accumulation steps = 1
  Total optimization steps = 7870


Step,Training Loss
500,0.086600
1000,0.115300
1500,0.123400
2000,0.119700
2500,0.115600
3000,0.106300
3500,0.095200
4000,0.090500
4500,0.080600
5000,0.075400


Saving model checkpoint to /content/drive/MyDrive/german-gpt2/checkpoint-800
Configuration saved in /content/drive/MyDrive/german-gpt2/checkpoint-800/config.json
Model weights saved in /content/drive/MyDrive/german-gpt2/checkpoint-800/pytorch_model.bin
Saving model checkpoint to /content/drive/MyDrive/german-gpt2/checkpoint-1600
Configuration saved in /content/drive/MyDrive/german-gpt2/checkpoint-1600/config.json
Model weights saved in /content/drive/MyDrive/german-gpt2/checkpoint-1600/pytorch_model.bin
Saving model checkpoint to /content/drive/MyDrive/german-gpt2/checkpoint-2400
Configuration saved in /content/drive/MyDrive/german-gpt2/checkpoint-2400/config.json
Model weights saved in /content/drive/MyDrive/german-gpt2/checkpoint-2400/pytorch_model.bin
Saving model checkpoint to /content/drive/MyDrive/german-gpt2/checkpoint-3200
Configuration saved in /content/drive/MyDrive/german-gpt2/checkpoint-3200/config.json
Model weights saved in /content/drive/MyDrive/german-gpt2/checkpoint-32

Step,Training Loss
500,0.086600
1000,0.115300
1500,0.123400
2000,0.119700
2500,0.115600
3000,0.106300
3500,0.095200
4000,0.090500
4500,0.080600
5000,0.075400


Saving model checkpoint to /content/drive/MyDrive/german-gpt2/checkpoint-7200
Configuration saved in /content/drive/MyDrive/german-gpt2/checkpoint-7200/config.json
Model weights saved in /content/drive/MyDrive/german-gpt2/checkpoint-7200/pytorch_model.bin


Training completed. Do not forget to share your model on huggingface.co/models =)




TrainOutput(global_step=7870, training_loss=0.08658960189722394, metrics={'train_runtime': 2660.2716, 'train_samples_per_second': 94.626, 'train_steps_per_second': 2.958, 'total_flos': 1.644376080384e+16, 'train_loss': 0.08658960189722394, 'epoch': 10.0})

In [ ]:
gpt2_trainer.train()

/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:310: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,
***** Running training *****
  Num examples = 25173
  Num Epochs = 50
  Instantaneous batch size per device = 32
  Total train batch size (w. parallel, distributed & accumulation) = 32
  Gradient Accumulation steps = 1
  Total optimization steps = 39350


Step,Training Loss
500,0.478200
1000,0.522300
1500,0.539100
2000,0.520000
2500,0.517500
3000,0.488700
3500,0.457100
4000,0.446300
4500,0.405500
5000,0.389700


Saving model checkpoint to /content/drive/MyDrive/german-gpt2/checkpoint-800
Configuration saved in /content/drive/MyDrive/german-gpt2/checkpoint-800/config.json
Model weights saved in /content/drive/MyDrive/german-gpt2/checkpoint-800/pytorch_model.bin
Saving model checkpoint to /content/drive/MyDrive/german-gpt2/checkpoint-1600
Configuration saved in /content/drive/MyDrive/german-gpt2/checkpoint-1600/config.json
Model weights saved in /content/drive/MyDrive/german-gpt2/checkpoint-1600/pytorch_model.bin
Saving model checkpoint to /content/drive/MyDrive/german-gpt2/checkpoint-2400
Configuration saved in /content/drive/MyDrive/german-gpt2/checkpoint-2400/config.json
Model weights saved in /content/drive/MyDrive/german-gpt2/checkpoint-2400/pytorch_model.bin
Saving model checkpoint to /content/drive/MyDrive/german-gpt2/checkpoint-3200
Configuration saved in /content/drive/MyDrive/german-gpt2/checkpoint-3200/config.json
Model weights saved in /content/drive/MyDrive/german-gpt2/checkpoint-32

KeyboardInterrupt: ignored

In [ ]:
gpt2_trainer.train()

/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:310: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,
***** Running training *****
  Num examples = 25173
  Num Epochs = 10
  Instantaneous batch size per device = 32
  Total train batch size (w. parallel, distributed & accumulation) = 32
  Gradient Accumulation steps = 1
  Total optimization steps = 7870


Step,Training Loss
500,0.582300
1000,0.692200
1500,0.728200
2000,0.689900
2500,0.670200
3000,0.611300
3500,0.559700
4000,0.530900
4500,0.474000
5000,0.448200


Saving model checkpoint to /content/drive/MyDrive/german-gpt2/checkpoint-800
Configuration saved in /content/drive/MyDrive/german-gpt2/checkpoint-800/config.json
Model weights saved in /content/drive/MyDrive/german-gpt2/checkpoint-800/pytorch_model.bin
Saving model checkpoint to /content/drive/MyDrive/german-gpt2/checkpoint-1600
Configuration saved in /content/drive/MyDrive/german-gpt2/checkpoint-1600/config.json
Model weights saved in /content/drive/MyDrive/german-gpt2/checkpoint-1600/pytorch_model.bin
Saving model checkpoint to /content/drive/MyDrive/german-gpt2/checkpoint-2400
Configuration saved in /content/drive/MyDrive/german-gpt2/checkpoint-2400/config.json
Model weights saved in /content/drive/MyDrive/german-gpt2/checkpoint-2400/pytorch_model.bin
Saving model checkpoint to /content/drive/MyDrive/german-gpt2/checkpoint-3200
Configuration saved in /content/drive/MyDrive/german-gpt2/checkpoint-3200/config.json
Model weights saved in /content/drive/MyDrive/german-gpt2/checkpoint-32

TrainOutput(global_step=7870, training_loss=0.5129303230566779, metrics={'train_runtime': 2032.3708, 'train_samples_per_second': 123.86, 'train_steps_per_second': 3.872, 'total_flos': 1.644376080384e+16, 'train_loss': 0.5129303230566779, 'epoch': 10.0})

In [ ]:
gpt2_trainer.train()

/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:310: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,
***** Running training *****
  Num examples = 25173
  Num Epochs = 50
  Instantaneous batch size per device = 32
  Total train batch size (w. parallel, distributed & accumulation) = 32
  Gradient Accumulation steps = 1
  Total optimization steps = 39350


Step,Training Loss
500,0.582300
1000,0.696800
1500,0.744700
2000,0.724500
2500,0.718700
3000,0.672300
3500,0.627300
4000,0.610400
4500,0.555300
5000,0.532600


Saving model checkpoint to /content/drive/MyDrive/german-gpt2/checkpoint-800
Configuration saved in /content/drive/MyDrive/german-gpt2/checkpoint-800/config.json
Model weights saved in /content/drive/MyDrive/german-gpt2/checkpoint-800/pytorch_model.bin
Saving model checkpoint to /content/drive/MyDrive/german-gpt2/checkpoint-1600
Configuration saved in /content/drive/MyDrive/german-gpt2/checkpoint-1600/config.json
Model weights saved in /content/drive/MyDrive/german-gpt2/checkpoint-1600/pytorch_model.bin
Saving model checkpoint to /content/drive/MyDrive/german-gpt2/checkpoint-2400
Configuration saved in /content/drive/MyDrive/german-gpt2/checkpoint-2400/config.json
Model weights saved in /content/drive/MyDrive/german-gpt2/checkpoint-2400/pytorch_model.bin
Saving model checkpoint to /content/drive/MyDrive/german-gpt2/checkpoint-3200
Configuration saved in /content/drive/MyDrive/german-gpt2/checkpoint-3200/config.json
Model weights saved in /content/drive/MyDrive/german-gpt2/checkpoint-32

Step,Training Loss
500,0.582300
1000,0.696800
1500,0.744700
2000,0.724500
2500,0.718700
3000,0.672300
3500,0.627300
4000,0.610400
4500,0.555300
5000,0.532600


Saving model checkpoint to /content/drive/MyDrive/german-gpt2/checkpoint-14400
Configuration saved in /content/drive/MyDrive/german-gpt2/checkpoint-14400/config.json
Model weights saved in /content/drive/MyDrive/german-gpt2/checkpoint-14400/pytorch_model.bin
Saving model checkpoint to /content/drive/MyDrive/german-gpt2/checkpoint-15200
Configuration saved in /content/drive/MyDrive/german-gpt2/checkpoint-15200/config.json
Model weights saved in /content/drive/MyDrive/german-gpt2/checkpoint-15200/pytorch_model.bin
Saving model checkpoint to /content/drive/MyDrive/german-gpt2/checkpoint-16000
Configuration saved in /content/drive/MyDrive/german-gpt2/checkpoint-16000/config.json
Model weights saved in /content/drive/MyDrive/german-gpt2/checkpoint-16000/pytorch_model.bin
Saving model checkpoint to /content/drive/MyDrive/german-gpt2/checkpoint-16800
Configuration saved in /content/drive/MyDrive/german-gpt2/checkpoint-16800/config.json
Model weights saved in /content/drive/MyDrive/german-gpt2

KeyboardInterrupt: ignored

In [ ]:
gpt2_trainer.train()

/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:310: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,
***** Running training *****
  Num examples = 25173
  Num Epochs = 50
  Instantaneous batch size per device = 32
  Total train batch size (w. parallel, distributed & accumulation) = 32
  Gradient Accumulation steps = 1
  Total optimization steps = 39350


Step,Training Loss
500,0.582300
1000,0.696900
1500,0.744400
2000,0.725100
2500,0.718700
3000,0.672100
3500,0.630100
4000,0.610300
4500,0.553700
5000,0.532800


Saving model checkpoint to /content/drive/MyDrive/german-gpt2/checkpoint-800
Configuration saved in /content/drive/MyDrive/german-gpt2/checkpoint-800/config.json
Model weights saved in /content/drive/MyDrive/german-gpt2/checkpoint-800/pytorch_model.bin
Saving model checkpoint to /content/drive/MyDrive/german-gpt2/checkpoint-1600
Configuration saved in /content/drive/MyDrive/german-gpt2/checkpoint-1600/config.json
Model weights saved in /content/drive/MyDrive/german-gpt2/checkpoint-1600/pytorch_model.bin
Saving model checkpoint to /content/drive/MyDrive/german-gpt2/checkpoint-2400
Configuration saved in /content/drive/MyDrive/german-gpt2/checkpoint-2400/config.json
Model weights saved in /content/drive/MyDrive/german-gpt2/checkpoint-2400/pytorch_model.bin
Saving model checkpoint to /content/drive/MyDrive/german-gpt2/checkpoint-3200
Configuration saved in /content/drive/MyDrive/german-gpt2/checkpoint-3200/config.json
Model weights saved in /content/drive/MyDrive/german-gpt2/checkpoint-32

KeyboardInterrupt: ignored

In [ ]:
gpt2_trainer.train()

***** Running training *****
  Num examples = 25173
  Num Epochs = 50
  Instantaneous batch size per device = 32
  Total train batch size (w. parallel, distributed & accumulation) = 32
  Gradient Accumulation steps = 1
  Total optimization steps = 39350


Step,Training Loss
500,2.797000
1000,2.765700
1500,2.723800
2000,2.652700
2500,2.616000
3000,2.560100
3500,2.491700
4000,2.474500
4500,2.386300
5000,2.344000


Saving model checkpoint to dbmdz/german-gpt2/checkpoint-800
Configuration saved in dbmdz/german-gpt2/checkpoint-800/config.json
Model weights saved in dbmdz/german-gpt2/checkpoint-800/pytorch_model.bin
Saving model checkpoint to dbmdz/german-gpt2/checkpoint-1600
Configuration saved in dbmdz/german-gpt2/checkpoint-1600/config.json
Model weights saved in dbmdz/german-gpt2/checkpoint-1600/pytorch_model.bin
Saving model checkpoint to dbmdz/german-gpt2/checkpoint-2400
Configuration saved in dbmdz/german-gpt2/checkpoint-2400/config.json
Model weights saved in dbmdz/german-gpt2/checkpoint-2400/pytorch_model.bin
Saving model checkpoint to dbmdz/german-gpt2/checkpoint-3200
Configuration saved in dbmdz/german-gpt2/checkpoint-3200/config.json
Model weights saved in dbmdz/german-gpt2/checkpoint-3200/pytorch_model.bin
Saving model checkpoint to dbmdz/german-gpt2/checkpoint-4000
Configuration saved in dbmdz/german-gpt2/checkpoint-4000/config.json
Model weights saved in dbmdz/german-gpt2/checkpoint-4

Step,Training Loss
500,2.797000
1000,2.765700
1500,2.723800
2000,2.652700
2500,2.616000
3000,2.560100
3500,2.491700
4000,2.474500
4500,2.386300
5000,2.344000


Saving model checkpoint to dbmdz/german-gpt2/checkpoint-10400
Configuration saved in dbmdz/german-gpt2/checkpoint-10400/config.json
Model weights saved in dbmdz/german-gpt2/checkpoint-10400/pytorch_model.bin
Saving model checkpoint to dbmdz/german-gpt2/checkpoint-11200
Configuration saved in dbmdz/german-gpt2/checkpoint-11200/config.json
Model weights saved in dbmdz/german-gpt2/checkpoint-11200/pytorch_model.bin
Saving model checkpoint to dbmdz/german-gpt2/checkpoint-12000
Configuration saved in dbmdz/german-gpt2/checkpoint-12000/config.json
Model weights saved in dbmdz/german-gpt2/checkpoint-12000/pytorch_model.bin
Saving model checkpoint to dbmdz/german-gpt2/checkpoint-12800
Configuration saved in dbmdz/german-gpt2/checkpoint-12800/config.json
Model weights saved in dbmdz/german-gpt2/checkpoint-12800/pytorch_model.bin
Saving model checkpoint to dbmdz/german-gpt2/checkpoint-13600
Configuration saved in dbmdz/german-gpt2/checkpoint-13600/config.json
Model weights saved in dbmdz/german-

KeyboardInterrupt: ignored

In [ ]:
gpt2_trainer.train()

***** Running training *****
  Num examples = 25173
  Num Epochs = 50
  Instantaneous batch size per device = 32
  Total train batch size (w. parallel, distributed & accumulation) = 32
  Gradient Accumulation steps = 1
  Total optimization steps = 39350


Step,Training Loss
500,3.435400
1000,3.350300
1500,3.267700
2000,3.181800
2500,3.132800
3000,3.075000
3500,3.011400
4000,2.976000
4500,2.909400
5000,2.858300


Saving model checkpoint to dbmdz/german-gpt2/checkpoint-800
Configuration saved in dbmdz/german-gpt2/checkpoint-800/config.json
Model weights saved in dbmdz/german-gpt2/checkpoint-800/pytorch_model.bin
Saving model checkpoint to dbmdz/german-gpt2/checkpoint-1600
Configuration saved in dbmdz/german-gpt2/checkpoint-1600/config.json
Model weights saved in dbmdz/german-gpt2/checkpoint-1600/pytorch_model.bin
Saving model checkpoint to dbmdz/german-gpt2/checkpoint-2400
Configuration saved in dbmdz/german-gpt2/checkpoint-2400/config.json
Model weights saved in dbmdz/german-gpt2/checkpoint-2400/pytorch_model.bin
Saving model checkpoint to dbmdz/german-gpt2/checkpoint-3200
Configuration saved in dbmdz/german-gpt2/checkpoint-3200/config.json
Model weights saved in dbmdz/german-gpt2/checkpoint-3200/pytorch_model.bin
Saving model checkpoint to dbmdz/german-gpt2/checkpoint-4000
Configuration saved in dbmdz/german-gpt2/checkpoint-4000/config.json
Model weights saved in dbmdz/german-gpt2/checkpoint-4

KeyboardInterrupt: ignored

In [ ]:
gpt2_trainer.train()

***** Running training *****
  Num examples = 25173
  Num Epochs = 50
  Instantaneous batch size per device = 32
  Total train batch size (w. parallel, distributed & accumulation) = 32
  Gradient Accumulation steps = 1
  Total optimization steps = 39350


Step,Training Loss
500,4.086900
1000,3.554600
1500,3.424900
2000,3.305800
2500,3.245500
3000,3.173300
3500,3.109500
4000,3.063000
4500,2.994000
5000,2.945600


Saving model checkpoint to /content/drive/MyDrive/german-gpt2/checkpoint-800
Configuration saved in /content/drive/MyDrive/german-gpt2/checkpoint-800/config.json
Model weights saved in /content/drive/MyDrive/german-gpt2/checkpoint-800/pytorch_model.bin
Saving model checkpoint to /content/drive/MyDrive/german-gpt2/checkpoint-1600
Configuration saved in /content/drive/MyDrive/german-gpt2/checkpoint-1600/config.json
Model weights saved in /content/drive/MyDrive/german-gpt2/checkpoint-1600/pytorch_model.bin
Saving model checkpoint to /content/drive/MyDrive/german-gpt2/checkpoint-2400
Configuration saved in /content/drive/MyDrive/german-gpt2/checkpoint-2400/config.json
Model weights saved in /content/drive/MyDrive/german-gpt2/checkpoint-2400/pytorch_model.bin
Saving model checkpoint to /content/drive/MyDrive/german-gpt2/checkpoint-3200
Configuration saved in /content/drive/MyDrive/german-gpt2/checkpoint-3200/config.json
Model weights saved in /content/drive/MyDrive/german-gpt2/checkpoint-32

Step,Training Loss
500,4.086900
1000,3.554600
1500,3.424900
2000,3.305800
2500,3.245500
3000,3.173300
3500,3.109500
4000,3.063000
4500,2.994000
5000,2.945600


Saving model checkpoint to /content/drive/MyDrive/german-gpt2/checkpoint-13600
Configuration saved in /content/drive/MyDrive/german-gpt2/checkpoint-13600/config.json
Model weights saved in /content/drive/MyDrive/german-gpt2/checkpoint-13600/pytorch_model.bin
Saving model checkpoint to /content/drive/MyDrive/german-gpt2/checkpoint-14400
Configuration saved in /content/drive/MyDrive/german-gpt2/checkpoint-14400/config.json
Model weights saved in /content/drive/MyDrive/german-gpt2/checkpoint-14400/pytorch_model.bin
Saving model checkpoint to /content/drive/MyDrive/german-gpt2/checkpoint-15200
Configuration saved in /content/drive/MyDrive/german-gpt2/checkpoint-15200/config.json
Model weights saved in /content/drive/MyDrive/german-gpt2/checkpoint-15200/pytorch_model.bin
Saving model checkpoint to /content/drive/MyDrive/german-gpt2/checkpoint-16000
Configuration saved in /content/drive/MyDrive/german-gpt2/checkpoint-16000/config.json
Model weights saved in /content/drive/MyDrive/german-gpt2

TrainOutput(global_step=39350, training_loss=1.619122526951636, metrics={'train_runtime': 10161.257, 'train_samples_per_second': 123.868, 'train_steps_per_second': 3.873, 'total_flos': 8.22188040192e+16, 'train_loss': 1.619122526951636, 'epoch': 50.0})

In [ ]:
model_path = dir + model_name

gpt2_trainer.save_model(model_path)

gpt2_pipe = pipeline("text-generation", model=model_path)

Saving model checkpoint to /content/drive/MyDrive/german-gpt2
Configuration saved in /content/drive/MyDrive/german-gpt2/config.json
Model weights saved in /content/drive/MyDrive/german-gpt2/pytorch_model.bin
loading configuration file /content/drive/MyDrive/german-gpt2/config.json
Model config GPT2Config {
  "_name_or_path": "/content/drive/MyDrive/german-gpt2",
  "activation_function": "gelu_new",
  "architectures": [
    "GPT2LMHeadModel"
  ],
  "attn_pdrop": 0.0,
  "bos_token_id": 50256,
  "embd_pdrop": 0.0,
  "eos_token_id": 50256,
  "gradient_checkpointing": false,
  "initializer_range": 0.02,
  "layer_norm_epsilon": 1e-05,
  "model_type": "gpt2",
  "n_ctx": 1024,
  "n_embd": 768,
  "n_head": 12,
  "n_inner": null,
  "n_layer": 12,
  "n_positions": 1024,
  "reorder_and_upcast_attn": false,
  "resid_pdrop": 0.0,
  "scale_attn_by_inverse_layer_idx": false,
  "scale_attn_weights": true,
  "summary_activation": null,
  "summary_first_dropout": 0.1,
  "summary_proj_to_labels": true,
  

In [ ]:
gpt2_pipe = pipeline("text-generation", model=model_path)

loading configuration file /content/drive/MyDrive/german-gpt2/config.json
Model config GPT2Config {
  "_name_or_path": "/content/drive/MyDrive/german-gpt2",
  "activation_function": "gelu_new",
  "architectures": [
    "GPT2LMHeadModel"
  ],
  "attn_pdrop": 0.0,
  "bos_token_id": 50256,
  "embd_pdrop": 0.0,
  "eos_token_id": 50256,
  "gradient_checkpointing": false,
  "initializer_range": 0.02,
  "layer_norm_epsilon": 1e-05,
  "model_type": "gpt2",
  "n_ctx": 1024,
  "n_embd": 768,
  "n_head": 12,
  "n_inner": null,
  "n_layer": 12,
  "n_positions": 1024,
  "reorder_and_upcast_attn": false,
  "resid_pdrop": 0.0,
  "scale_attn_by_inverse_layer_idx": false,
  "scale_attn_weights": true,
  "summary_activation": null,
  "summary_first_dropout": 0.1,
  "summary_proj_to_labels": true,
  "summary_type": "cls_index",
  "summary_use_proj": true,
  "task_specific_params": {
    "text-generation": {
      "do_sample": true,
      "max_length": 50
    }
  },
  "torch_dtype": "float32",
  "transfor

Zunächst kann ich sagen, dass deine erste Aufgabe gut gelöst wurde. Du hast die Umwelt- und Unternehmensanalyse gut strukturiert aufgebaut, eine gute Einleitung geschrieben und die einzelnen Fakten auf die verschiedenen Faktoren bezogen, auch die zu hohen Produktionskosten hast du raus gearbeitet. Dass du das Unternehmen noch einmal aus der innen-außen-Perspektive, sowie anders rum betrachtet hast ist noch einmal in die etwas in die Tiefe gegangen. Bis auf, dass du vielleicht noch die Kommunikationsprobleme unter den einzelnen Abteilungen hättest ansprechen können, habe ich keine Anmerkungen zur ersten Aufgabe.in der zweiten Aufgabe hast du die wichtigsten Fehler angesprochen und deren Folgen benannt. Hier hätte ma vielleicht einzelne Aspekte noch näher erläutert können, jedoch halte ich dies nicht für absolut notwendig, da die Frage beantwortet wurde."


In [ ]:
gpt2_pipe = pipeline("text-generation", model=model_path)
gpt2_pipe("Zunächst kann ich sagen, dass deine erste Aufgabe gut gelöst wurde. Du hast die Umwelt- und Unternehmensanalyse gut strukturiert aufgebaut, eine gute Einleitung geschrieben und die einzelnen Fakten auf die verschiedenen Faktoren bezogen, auch die zu hohen Produktionskosten hast du raus gearbeitet.", max_length=150)[0]["generated_text"]

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


'Zunächst kann ich sagen, dass deine erste Aufgabe gut gelöst wurde. Du hast die Umwelt- und Unternehmensanalyse gut strukturiert aufgebaut, eine gute Einleitung geschrieben und die einzelnen Fakten auf die verschiedenen Faktoren bezogen, auch die zu hohen Produktionskosten hast du raus gearbeitet. Lediglich die Veränderungen bei den neuen politisch-rechsextremen Bewegungen,, hättest du nochmal genauer betrachten können. Zum Schluss hättest du dein abgeschlossenes, durch konstruktives und detaillierte Arbeitgeist-Denken würdigen und in der Realität umsetzen können. Ich finde es sehr gut, dass du von eine Umwelt/rech Ostens, eine zur anderen gedacht hast. Das Problem, dies sieht man auch, wurde einfach ignoriert.Allerdings wurde es dort schwer konkreter zu werden und man hätte ein wenig mehr in die Tiefe denken können. Insgesamt aber ein'

In [ ]:
gpt2_pipe("Zunächst kann ich sagen, dass deine erste Aufgabe gut gelöst wurde. Du hast die Umwelt- und Unternehmensanalyse gut strukturiert aufgebaut, eine gute Einleitung geschrieben und die einzelnen Fakten auf die verschiedenen Faktoren bezogen, auch die zu hohen Produktionskosten hast du raus gearbeitet.", max_length=150)[0]["generated_text"]

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


'Zunächst kann ich sagen, dass deine erste Aufgabe gut gelöst wurde. Du hast die Umwelt- und Unternehmensanalyse gut strukturiert aufgebaut, eine gute Einleitung geschrieben und die einzelnen Fakten auf die verschiedenen Faktoren bezogen, auch die zu hohen Produktionskosten hast du raus gearbeitet. Bei der Umweltanalyse würde ich zusätzlich sagen, dass die Zahlungshemmnisse den Grund für die Kostenreduzierung sind. Um die sinkenden Einnahmen und die dadurch entstehen Schäden etwas auszugleichen könnten höhere Zuzahlungen fällig gestellt werden. Zusätzlich gibt es noch die Frage der zu hohen Produktionskosten in der zu hohen Fehlerrate. Ich finde es ist gut das du Bezug auf die Problematik nimmst, jedoch triffst du keine konkreten Taten in der Aufgabe 2\n\nMir gefällt deine kurze und fast schon zu lang ausgearbeitete Detailansicht sehr'

In [ ]:
text

'Zunächst kann ich sagen, dass deine erste Aufgabe gut gelöst wurde. Du gehst zuerst auf die Wettbewerbsstrategie ein und stellst dann richtige Wege dar, aber du gehst auch auf die beiden Seiten ein (Bspw. Lieferanten und Markt) und bearbeitest deren tatsächliches Handeln in dem jeweiligen Schritt (damit meine ich Kostenanalysen, etc.). Hier scheinst du alle Konfliktfelder zu endeverdinen und zu bei der letzten Aufgabe solltest du nochmal zu einer differenzierteren Beurteilung kommen. Ich nehme leider deine Lösungsvorschläge an und kann für meine eigene Lösung nur noch dazu sagen: Hätte nicht jeder der hier ein ist, wirklich in dem Wettbewerb arbeiten müssen? Vielleicht hättest du ein- oder andere offensichtliche istichtigungsprogramme haben können, du hättest Zusammenhänge erkennen'

In [ ]:
text = gpt2_pipe("Zunächst kann ich sagen, dass deine erste Aufgabe gut gelöst wurde.", max_length=150)[0]["generated_text"]

print(text)

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Zunächst kann ich sagen, dass deine erste Aufgabe gut gelöst wurde. Du bist sehr strukturiert, genau und verständlich auf die Aufgaben eingegangen. Es wurden mehrere Berührungspunkte angesprochen und dementsprechend wurden Fachbegriffe verwendet. Der Bezug zum Sachverhalt, sowie seine Erklärung ist sehr gelungen. Der Leser ist natürlich immer bei deiner Beurteilung und deiner Wahl der Abschlusstest!!

Ich finde deine Geschäftsidee sehr spannend und sie ist interessant. Du verwendest im Service Blueprint und im  alle 10 10 Informationssysteme (Studentenhandbuch und stellenc02_stueck kasten_0 ist toll eingefügtz. Komm!z.Bei  hast du sehr viele Interfaces, bedeutende Symbole und Instandsätze fokusiert an der Erstellung stilechter Grafik


In [ ]:
german_gpt2_model = AutoModelWithLMHead.from_pretrained(model_path)

/usr/local/lib/python3.7/dist-packages/transformers/models/auto/modeling_auto.py:1066: FutureWarning: The class `AutoModelWithLMHead` is deprecated and will be removed in a future version. Please use `AutoModelForCausalLM` for causal language models, `AutoModelForMaskedLM` for masked language models and `AutoModelForSeq2SeqLM` for encoder-decoder models.
  FutureWarning,


In [ ]:
training_args = TrainingArguments(
    output_dir = model_path,
    overwrite_output_dir = True,
    num_train_epochs = 10,
    per_device_train_batch_size = 32,
    per_device_eval_batch_size = 64,
    eval_steps = 400,
    save_steps = 800,
    warmup_steps = 500,
)
gpt2_trainer = Trainer(
    model = german_gpt2_model,
    args = training_args,
    data_collator = data_collator,
    train_dataset = train_dataset,
    eval_dataset = test_dataset,
    # compute_metrics = compute_metrics
    # prediction_loss_only = True,
)

In [ ]:
gpt2_trainer.train()

***** Running training *****
  Num examples = 25158
  Num Epochs = 10
  Instantaneous batch size per device = 32
  Total train batch size (w. parallel, distributed & accumulation) = 32
  Gradient Accumulation steps = 1
  Total optimization steps = 7870


Step,Training Loss
500,1.050200
1000,1.129200
1500,1.154900
2000,1.101500
2500,1.064700
3000,0.999000
3500,0.931000
4000,0.899900
4500,0.825400
5000,0.788500


Saving model checkpoint to /content/drive/MyDrive/Colab Notebooks/Su/german-gpt2/checkpoint-800
Configuration saved in /content/drive/MyDrive/Colab Notebooks/Su/german-gpt2/checkpoint-800/config.json
Model weights saved in /content/drive/MyDrive/Colab Notebooks/Su/german-gpt2/checkpoint-800/pytorch_model.bin
Saving model checkpoint to /content/drive/MyDrive/Colab Notebooks/Su/german-gpt2/checkpoint-1600
Configuration saved in /content/drive/MyDrive/Colab Notebooks/Su/german-gpt2/checkpoint-1600/config.json
Model weights saved in /content/drive/MyDrive/Colab Notebooks/Su/german-gpt2/checkpoint-1600/pytorch_model.bin
Saving model checkpoint to /content/drive/MyDrive/Colab Notebooks/Su/german-gpt2/checkpoint-2400
Configuration saved in /content/drive/MyDrive/Colab Notebooks/Su/german-gpt2/checkpoint-2400/config.json
Model weights saved in /content/drive/MyDrive/Colab Notebooks/Su/german-gpt2/checkpoint-2400/pytorch_model.bin
Saving model checkpoint to /content/drive/MyDrive/Colab Notebook

TrainOutput(global_step=7870, training_loss=0.8762647345226632, metrics={'train_runtime': 2089.1921, 'train_samples_per_second': 120.42, 'train_steps_per_second': 3.767, 'total_flos': 1.643396235264e+16, 'train_loss': 0.8762647345226632, 'epoch': 10.0})

### bert-base-german-cased

In [ ]:
model_name = 'bert-base-german-cased'
model_path = 'bertgerman/'

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(model_name)

train_dataset, test_dataset, data_collator = load_dataset(train_path, test_path, tokenizer)

Downloading:   0%|          | 0.00/29.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/433 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/255k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/485k [00:00<?, ?B/s]

/usr/local/lib/python3.7/dist-packages/transformers/data/datasets/language_modeling.py:58: FutureWarning: This dataset will be removed from the library soon, preprocessing should be handled with the 🤗 Datasets library. You can have a look at this example script for pointers: https://github.com/huggingface/transformers/blob/main/examples/pytorch/language-modeling/run_mlm.py
  FutureWarning,


In [ ]:
bert_german_model = AutoModelWithLMHead.from_pretrained(model_name)
# tokenizer.save_pretrained(model_path)

/usr/local/lib/python3.7/dist-packages/transformers/models/auto/modeling_auto.py:1116: FutureWarning: The class `AutoModelWithLMHead` is deprecated and will be removed in a future version. Please use `AutoModelForCausalLM` for causal language models, `AutoModelForMaskedLM` for masked language models and `AutoModelForSeq2SeqLM` for encoder-decoder models.
  FutureWarning,


Downloading:   0%|          | 0.00/439M [00:00<?, ?B/s]

Some weights of the model checkpoint at bert-base-german-cased were not used when initializing BertForMaskedLM: ['cls.seq_relationship.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [ ]:
training_args = TrainingArguments(
    output_dir = model_path,
    overwrite_output_dir = True,
    num_train_epochs = 50,
    per_device_train_batch_size = 32,
    per_device_eval_batch_size = 64,
    eval_steps = 400,
    save_steps = 800,
    warmup_steps = 500,
)
bert_trainer = Trainer(
    model = bert_german_model,
    args = training_args,
    data_collator = data_collator,
    train_dataset = train_dataset,
    eval_dataset = test_dataset,
    # compute_metrics = compute_metrics
    # prediction_loss_only = True,
)

In [ ]:
bert_trainer.train()

/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:310: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,
***** Running training *****
  Num examples = 27529
  Num Epochs = 50
  Instantaneous batch size per device = 32
  Total train batch size (w. parallel, distributed & accumulation) = 32
  Gradient Accumulation steps = 1
  Total optimization steps = 43050


Step,Training Loss


In [ ]:
bert_trainer.save_model(model_path)

bert_pipe = pipeline("text-generation", model=model_path)

Saving model checkpoint to /content/drive/MyDrive/bert-german
Configuration saved in /content/drive/MyDrive/bert-german/config.json
Model weights saved in /content/drive/MyDrive/bert-german/pytorch_model.bin
loading configuration file /content/drive/MyDrive/bert-german/config.json
Model config BertConfig {
  "_name_or_path": "/content/drive/MyDrive/bert-german",
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "torch_dtype": "float32",
  "transformers_version": "4.23.1",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 30000
}

loading configuration file /content/drive/MyDri

In [ ]:
text = bert_pipe("Zunächst kann ich sagen, dass deine erste Aufgabe gut gelöst wurde.", max_length=150)[0]["generated_text"]

print(text)

Zunächst kann ich sagen, dass deine erste Aufgabe gut gelöst wurde..........................................................................................................................................


In [5]:
model_name = 'benjamin/gerpt2'
model_path = 'gerpt2'

tokenizer = AutoTokenizer.from_pretrained(model_name)

In [6]:
train_dataset, test_dataset, data_collator = load_dataset(train_path, test_path, tokenizer)

In [7]:
gerpt2_model = AutoModelWithLMHead.from_pretrained(model_name)


/opt/conda/lib/python3.9/site-packages/transformers/models/auto/modeling_auto.py:1177: FutureWarning: The class `AutoModelWithLMHead` is deprecated and will be removed in a future version. Please use `AutoModelForCausalLM` for causal language models, `AutoModelForMaskedLM` for masked language models and `AutoModelForSeq2SeqLM` for encoder-decoder models.
  warnings.warn(


Downloading:   0%|          | 0.00/665M [00:00<?, ?B/s]

In [8]:
training_args = TrainingArguments(
    output_dir = model_path,
    overwrite_output_dir = True,
    num_train_epochs = 20,
    per_device_train_batch_size = 16,
    per_device_eval_batch_size = 32,
    eval_steps = 400,
    save_steps = 800,
    warmup_steps = 500,
)
gerpt2_trainer = Trainer(
    model = gerpt2_model,
    args = training_args,
    data_collator = data_collator,
    train_dataset = train_dataset,
    eval_dataset = test_dataset,
    # compute_metrics = compute_metrics
    # prediction_loss_only = True,
)

In [9]:
gerpt2_trainer.train()

/opt/conda/lib/python3.9/site-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 20629
  Num Epochs = 20
  Instantaneous batch size per device = 16
  Total train batch size (w. parallel, distributed & accumulation) = 32
  Gradient Accumulation steps = 1
  Total optimization steps = 12900
  Number of trainable parameters = 163037184
/opt/conda/lib/python3.9/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '


Step,Training Loss
500,4.042400
1000,3.614400
1500,3.501000
2000,3.423900
2500,3.360700
3000,3.309500
3500,3.272000
4000,3.237600
4500,3.205300
5000,3.167800


Saving model checkpoint to gerpt2/checkpoint-800
Configuration saved in gerpt2/checkpoint-800/config.json
Model weights saved in gerpt2/checkpoint-800/pytorch_model.bin
/opt/conda/lib/python3.9/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '
Saving model checkpoint to gerpt2/checkpoint-1600
Configuration saved in gerpt2/checkpoint-1600/config.json
Model weights saved in gerpt2/checkpoint-1600/pytorch_model.bin
/opt/conda/lib/python3.9/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '
Saving model checkpoint to gerpt2/checkpoint-2400
Configuration saved in gerpt2/checkpoint-2400/config.json
Model

TrainOutput(global_step=12900, training_loss=3.1756474836482558, metrics={'train_runtime': 3747.1771, 'train_samples_per_second': 110.104, 'train_steps_per_second': 3.443, 'total_flos': 3.918098678022144e+16, 'train_loss': 3.1756474836482558, 'epoch': 20.0})

In [12]:
gerpt2_trainer.train()

***** Running training *****
  Num examples = 20629
  Num Epochs = 20
  Instantaneous batch size per device = 16
  Total train batch size (w. parallel, distributed & accumulation) = 32
  Gradient Accumulation steps = 1
  Total optimization steps = 12900
  Number of trainable parameters = 163037184


Step,Training Loss
500,2.951800
1000,2.951400
1500,2.958900
2000,2.950900
2500,2.954000
3000,2.953800
3500,2.952200
4000,2.954900
4500,2.953400
5000,2.954700


Saving model checkpoint to gerpt2/checkpoint-800
Configuration saved in gerpt2/checkpoint-800/config.json
Model weights saved in gerpt2/checkpoint-800/pytorch_model.bin
/opt/conda/lib/python3.9/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '
Saving model checkpoint to gerpt2/checkpoint-1600
Configuration saved in gerpt2/checkpoint-1600/config.json
Model weights saved in gerpt2/checkpoint-1600/pytorch_model.bin
/opt/conda/lib/python3.9/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '
Saving model checkpoint to gerpt2/checkpoint-2400
Configuration saved in gerpt2/checkpoint-2400/config.json
Model

TrainOutput(global_step=12900, training_loss=2.953411373167999, metrics={'train_runtime': 3730.3857, 'train_samples_per_second': 110.6, 'train_steps_per_second': 3.458, 'total_flos': 3.918098678022144e+16, 'train_loss': 2.953411373167999, 'epoch': 20.0})

In [13]:
!tar chvfz gerpt2-40.tar.gz 'gerpt2/checkpoint-12800'

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
gerpt2/checkpoint-12800/
gerpt2/checkpoint-12800/rng_state.pth
gerpt2/checkpoint-12800/optimizer.pt
gerpt2/checkpoint-12800/scheduler.pt
gerpt2/checkpoint-12800/config.json
gerpt2/checkpoint-12800/training_args.bin
gerpt2/checkpoint-12800/trainer_state.json
gerpt2/checkpoint-12800/pytorch_model.bin


In [11]:
!tar chvfz gerpt2-20.tar.gz 'gerpt2/checkpoint-12800'

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
gerpt2/checkpoint-12800/
gerpt2/checkpoint-12800/rng_state.pth
gerpt2/checkpoint-12800/optimizer.pt
gerpt2/checkpoint-12800/scheduler.pt
gerpt2/checkpoint-12800/config.json
gerpt2/checkpoint-12800/training_args.bin
gerpt2/checkpoint-12800/trainer_state.json
gerpt2/checkpoint-12800/pytorch_model.bin


In [17]:
tokenizer.save_pretrained(model_path)

tokenizer config file saved in gerpt2/tokenizer_config.json
Special tokens file saved in gerpt2/special_tokens_map.json


('gerpt2/tokenizer_config.json',
 'gerpt2/special_tokens_map.json',
 'gerpt2/vocab.json',
 'gerpt2/merges.txt',
 'gerpt2/added_tokens.json',
 'gerpt2/tokenizer.json')

In [14]:
gerpt2_trainer.save_model(model_path)

Saving model checkpoint to gerpt2
Configuration saved in gerpt2/config.json
Model weights saved in gerpt2/pytorch_model.bin
loading configuration file gerpt2/config.json
Model config GPT2Config {
  "_name_or_path": "gerpt2",
  "activation_function": "gelu_new",
  "architectures": [
    "GPT2LMHeadModel"
  ],
  "attn_pdrop": 0.1,
  "bos_token_id": 50256,
  "embd_pdrop": 0.1,
  "eos_token_id": 50256,
  "gradient_checkpointing": false,
  "initializer_range": 0.02,
  "layer_norm_epsilon": 1e-05,
  "model_type": "gpt2",
  "n_ctx": 1024,
  "n_embd": 768,
  "n_head": 12,
  "n_inner": null,
  "n_layer": 12,
  "n_positions": 1024,
  "pad_token_id": 0,
  "reorder_and_upcast_attn": false,
  "resid_pdrop": 0.1,
  "scale_attn_by_inverse_layer_idx": false,
  "scale_attn_weights": true,
  "summary_activation": null,
  "summary_first_dropout": 0.1,
  "summary_proj_to_labels": true,
  "summary_type": "cls_index",
  "summary_use_proj": true,
  "task_specific_params": {
    "text-generation": {
      "do

OSError: Can't load tokenizer for 'gerpt2'. If you were trying to load it from 'https://huggingface.co/models', make sure you don't have a local directory with the same name. Otherwise, make sure 'gerpt2' is the correct path to a directory containing all relevant files for a GPT2TokenizerFast tokenizer.

In [18]:
gerpt2_pipe = pipeline("text-generation", model=model_path)

loading configuration file gerpt2/config.json
Model config GPT2Config {
  "_name_or_path": "gerpt2",
  "activation_function": "gelu_new",
  "architectures": [
    "GPT2LMHeadModel"
  ],
  "attn_pdrop": 0.1,
  "bos_token_id": 50256,
  "embd_pdrop": 0.1,
  "eos_token_id": 50256,
  "gradient_checkpointing": false,
  "initializer_range": 0.02,
  "layer_norm_epsilon": 1e-05,
  "model_type": "gpt2",
  "n_ctx": 1024,
  "n_embd": 768,
  "n_head": 12,
  "n_inner": null,
  "n_layer": 12,
  "n_positions": 1024,
  "pad_token_id": 0,
  "reorder_and_upcast_attn": false,
  "resid_pdrop": 0.1,
  "scale_attn_by_inverse_layer_idx": false,
  "scale_attn_weights": true,
  "summary_activation": null,
  "summary_first_dropout": 0.1,
  "summary_proj_to_labels": true,
  "summary_type": "cls_index",
  "summary_use_proj": true,
  "task_specific_params": {
    "text-generation": {
      "do_sample": true,
      "max_length": 100
    }
  },
  "tie_word_embeddings": false,
  "torch_dtype": "float32",
  "transforme

In [19]:
text = gerpt2_pipe("Zunächst kann ich sagen, dass deine erste Aufgabe gut gelöst wurde.", max_length=150)[0]["generated_text"]

print(text)

Zunächst kann ich sagen, dass deine erste Aufgabe gut gelöst wurde. Die Fragen wurden sehr gut auf den Punkt gebracht und deine Ausarbeitung war sehr durchdacht. Du konntest auch sehr gut auf die Begriffe der Umwelt- und Unternehmensanalyse eingehen. Auch deine zweite Aufgabe gefällt mir sehr gut. Hier hättest du noch ein wenig mehr auf die Frage der ökologischen Auswirkungen eingehen können. Jedoch hat mir deine Abgabe auch vollkommen gereicht, um zu verstehen, was die neuen Regeln zur Wettbewerbsumwelt sind. Am Ende deiner Ausarbeitung hätte ich noch eine kleine Bemerkung zum letzten Absatz. Ansonsten finde ich die Aufgabe sehr gelungen und wurde positiv überrascht unda heroffe ich konnte dir ein paar hilfreiche Tipps geben.

Die Lösung ist insgesamt sehr gut ausgearbeitet. Mir hat besonders gut gefallen, dass du den


In [20]:
root = "Zunächst kann ich sagen, dass deine erste Aufgabe gut gelöst wurde. Du hast die Umwelt- und Unternehmensanalyse gut strukturiert aufgebaut, eine gute Einleitung geschrieben und die einzelnen Fakten auf die verschiedenen Faktoren bezogen, auch die zu hohen Produktionskosten hast du raus gearbeitet."
text = gerpt2_pipe(root, max_length=150)[0]["generated_text"]

print(text)

Zunächst kann ich sagen, dass deine erste Aufgabe gut gelöst wurde. Du hast die Umwelt- und Unternehmensanalyse gut strukturiert aufgebaut, eine gute Einleitung geschrieben und die einzelnen Fakten auf die verschiedenen Faktoren bezogen, auch die zu hohen Produktionskosten hast du raus gearbeitet. Ich finde auch, dass du die wesentlichen Probleme in Deinem Unternehmen erkannt hast. Du hast alle wichtigen Probleme des Unternehmens genannt, allerdings fehlen meines Erachtens noch die Lösungsansätze des Konflikts, den du im ersten Absatz aufgezählt hast. Hier könntest du noch mehr auf das Problem eingehen, um das es geht und wie man dies beheben könnte, da ich sehe, dass die Aufgabe auf den ersten Blick etwas knapp und knapp ist. Außerdem könntest du zum Beispiel erwähnen wie die Strategie des Unternehmens bei der Wettbewerbsumwelt funktioniert und eine


### dbmdz/bert-base-german-cased

In [4]:
model_name = 'dbmdz/bert-base-german-cased'
model_path = 'dbgerman'
tokenizer = AutoTokenizer.from_pretrained(model_name)
train_dataset, test_dataset, data_collator = load_dataset(train_path, test_path, tokenizer)

Downloading:   0%|          | 0.00/59.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/456 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/240k [00:00<?, ?B/s]

/opt/conda/lib/python3.9/site-packages/transformers/data/datasets/language_modeling.py:54: FutureWarning: This dataset will be removed from the library soon, preprocessing should be handled with the 🤗 Datasets library. You can have a look at this example script for pointers: https://github.com/huggingface/transformers/blob/main/examples/pytorch/language-modeling/run_mlm.py
  warnings.warn(
Token indices sequence length is longer than the specified maximum sequence length for this model (2802217 > 512). Running this sequence through the model will result in indexing errors


In [5]:
dbgerman_model = AutoModelWithLMHead.from_pretrained(model_name)

/opt/conda/lib/python3.9/site-packages/transformers/models/auto/modeling_auto.py:1177: FutureWarning: The class `AutoModelWithLMHead` is deprecated and will be removed in a future version. Please use `AutoModelForCausalLM` for causal language models, `AutoModelForMaskedLM` for masked language models and `AutoModelForSeq2SeqLM` for encoder-decoder models.
  warnings.warn(


Downloading:   0%|          | 0.00/442M [00:00<?, ?B/s]

In [8]:
training_args = TrainingArguments(
    output_dir = model_path,
    overwrite_output_dir = True,
    num_train_epochs = 20,
    per_device_train_batch_size = 32,
    per_device_eval_batch_size = 64,
    eval_steps = 400,
    save_steps = 800,
    warmup_steps = 500,
)
dbgerman_trainer = Trainer(
    model = dbgerman_model,
    args = training_args,
    data_collator = data_collator,
    train_dataset = train_dataset,
    eval_dataset = test_dataset,
    # compute_metrics = compute_metrics
    # prediction_loss_only = True,
)

PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).


In [9]:
dbgerman_trainer.train()

***** Running training *****
  Num examples = 22239
  Num Epochs = 20
  Instantaneous batch size per device = 32
  Total train batch size (w. parallel, distributed & accumulation) = 64
  Gradient Accumulation steps = 1
  Total optimization steps = 6960
  Number of trainable parameters = 109960318


Step,Training Loss
500,0.000000
1000,0.000000


Saving model checkpoint to dbgerman/checkpoint-800
Configuration saved in dbgerman/checkpoint-800/config.json
Model weights saved in dbgerman/checkpoint-800/pytorch_model.bin
/opt/conda/lib/python3.9/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '


KeyboardInterrupt: 

In [11]:
model_path = 'dbgerman/checkpoint-800'
tokenizer.save_pretrained(model_path)
dbgerman_pipe = pipeline("text-generation", model=model_path)

tokenizer config file saved in dbgerman/checkpoint-800/tokenizer_config.json
Special tokens file saved in dbgerman/checkpoint-800/special_tokens_map.json
loading configuration file dbgerman/checkpoint-800/config.json
Model config BertConfig {
  "_name_or_path": "dbgerman/checkpoint-800",
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "output_past": true,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "torch_dtype": "float32",
  "transformers_version": "4.25.1",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 31102
}

loading configuration file dbgerman/checkpoint-800/config.json
Model config BertConfig {
  "_name_or

In [13]:
root = "Zunächst kann ich sagen, dass deine erste Aufgabe gut gelöst wurde. Du hast die Umwelt- und Unternehmensanalyse gut strukturiert aufgebaut, eine gute Einleitung geschrieben und die einzelnen Fakten auf die verschiedenen Faktoren bezogen, auch die zu hohen Produktionskosten hast du raus gearbeitet."
text = dbgerman_pipe(root, max_length=150)[0]["generated_text"]

print(text)

Zunächst kann ich sagen, dass deine erste Aufgabe gut gelöst wurde. Du hast die Umwelt- und Unternehmensanalyse gut strukturiert aufgebaut, eine gute Einleitung geschrieben und die einzelnen Fakten auf die verschiedenen Faktoren bezogen, auch die zu hohen Produktionskosten hast du raus gearbeitet....................................................................................................


### benjamin/gerpt2-large

In [4]:
model_name = 'benjamin/gerpt2-large'
model_path = 'gerpt2l/'

In [5]:
tokenizer = AutoTokenizer.from_pretrained(model_name)

train_dataset, test_dataset, data_collator = load_dataset(train_path, test_path, tokenizer)

/opt/conda/lib/python3.9/site-packages/transformers/data/datasets/language_modeling.py:54: FutureWarning: This dataset will be removed from the library soon, preprocessing should be handled with the 🤗 Datasets library. You can have a look at this example script for pointers: https://github.com/huggingface/transformers/blob/main/examples/pytorch/language-modeling/run_mlm.py
  warnings.warn(


In [7]:
gerpt2l_model = AutoModelWithLMHead.from_pretrained(model_name)
# tokenizer.save_pretrained(model_path)

/opt/conda/lib/python3.9/site-packages/transformers/models/auto/modeling_auto.py:1177: FutureWarning: The class `AutoModelWithLMHead` is deprecated and will be removed in a future version. Please use `AutoModelForCausalLM` for causal language models, `AutoModelForMaskedLM` for masked language models and `AutoModelForSeq2SeqLM` for encoder-decoder models.
  warnings.warn(


Downloading:   0%|          | 0.00/3.39G [00:00<?, ?B/s]

In [10]:
training_args = TrainingArguments(
    output_dir = model_path,
    overwrite_output_dir = True,
    num_train_epochs = 10,
    per_device_train_batch_size = 16,
    per_device_eval_batch_size = 32,
    eval_steps = 400,
    save_steps = 800,
    warmup_steps = 500,
)
gerpt2l_trainer = Trainer(
    model = gerpt2l_model,
    args = training_args,
    data_collator = data_collator,
    train_dataset = train_dataset,
    eval_dataset = test_dataset,
    # compute_metrics = compute_metrics
    # prediction_loss_only = True,
)

/opt/conda/lib/python3.9/site-packages/torch/cuda/__init__.py:145: UserWarning: 
NVIDIA A100-SXM4-40GB with CUDA capability sm_80 is not compatible with the current PyTorch installation.
The current PyTorch install supports CUDA capabilities sm_37 sm_50 sm_60 sm_70.
If you want to use the NVIDIA A100-SXM4-40GB GPU with PyTorch, please check the instructions at https://pytorch.org/get-started/locally/

  warnings.warn(incompatible_device_warn.format(device_name, capability, " ".join(arch_list), device_name))


In [9]:
gerpt2l_trainer.train()

/opt/conda/lib/python3.9/site-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 20629
  Num Epochs = 10
  Instantaneous batch size per device = 16
  Total train batch size (w. parallel, distributed & accumulation) = 32
  Gradient Accumulation steps = 1
  Total optimization steps = 6450
  Number of trainable parameters = 774030080
/opt/conda/lib/python3.9/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '


Step,Training Loss
500,3.627900
1000,3.230300
1500,3.098100
2000,2.987700
2500,2.879100
3000,2.785500
3500,2.720200
4000,2.660200
4500,2.602500
5000,2.543500


Saving model checkpoint to gerpt2l/checkpoint-800
Configuration saved in gerpt2l/checkpoint-800/config.json
Model weights saved in gerpt2l/checkpoint-800/pytorch_model.bin
/opt/conda/lib/python3.9/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '
Saving model checkpoint to gerpt2l/checkpoint-1600
Configuration saved in gerpt2l/checkpoint-1600/config.json
Model weights saved in gerpt2l/checkpoint-1600/pytorch_model.bin
/opt/conda/lib/python3.9/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '
Saving model checkpoint to gerpt2l/checkpoint-2400
Configuration saved in gerpt2l/checkpoint-2400/config.js

TrainOutput(global_step=6450, training_loss=2.818388993610707, metrics={'train_runtime': 7957.554, 'train_samples_per_second': 25.924, 'train_steps_per_second': 0.811, 'total_flos': 1.12230801113088e+17, 'train_loss': 2.818388993610707, 'epoch': 10.0})

In [ ]:
!tar chvfz gerpt2l.tar.gz 'gerpt2l/checkpoint-6400'

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
gerpt2l/checkpoint-6400/
gerpt2l/checkpoint-6400/pytorch_model.bin


In [6]:
!tar -xvzf gerpt2l-10.tar.gz

gerpt2l/checkpoint-6400/
gerpt2l/checkpoint-6400/pytorch_model.bin
gerpt2l/checkpoint-6400/optimizer.pt
gerpt2l/checkpoint-6400/trainer_state.json
gerpt2l/checkpoint-6400/scheduler.pt
gerpt2l/checkpoint-6400/rng_state.pth
gerpt2l/checkpoint-6400/training_args.bin
gerpt2l/checkpoint-6400/config.json


In [7]:
gerpt2l_model = AutoModelWithLMHead.from_pretrained('gerpt2l/checkpoint-6400')
# tokenizer.save_pretrained(model_path)

/opt/conda/lib/python3.9/site-packages/transformers/models/auto/modeling_auto.py:1177: FutureWarning: The class `AutoModelWithLMHead` is deprecated and will be removed in a future version. Please use `AutoModelForCausalLM` for causal language models, `AutoModelForMaskedLM` for masked language models and `AutoModelForSeq2SeqLM` for encoder-decoder models.
  warnings.warn(


In [11]:
model_path = 'gerpt2l/checkpoint-6400'
tokenizer = AutoTokenizer.from_pretrained(model_name)
tokenizer.save_pretrained(model_path)

('gerpt2l/checkpoint-6400/tokenizer_config.json',
 'gerpt2l/checkpoint-6400/special_tokens_map.json',
 'gerpt2l/checkpoint-6400/vocab.json',
 'gerpt2l/checkpoint-6400/merges.txt',
 'gerpt2l/checkpoint-6400/added_tokens.json',
 'gerpt2l/checkpoint-6400/tokenizer.json')

In [14]:
gerpt2l_pipe = pipeline("text-generation", model=model_path)

In [15]:
root = "Zunächst kann ich sagen, dass deine erste Aufgabe gut gelöst wurde. Du hast die Umwelt- und Unternehmensanalyse gut strukturiert aufgebaut, eine gute Einleitung geschrieben und die einzelnen Fakten auf die verschiedenen Faktoren bezogen, auch die zu hohen Produktionskosten hast du raus gearbeitet."
text = gerpt2l_pipe(root, max_length=150)[0]["generated_text"]

print(text)

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Zunächst kann ich sagen, dass deine erste Aufgabe gut gelöst wurde. Du hast die Umwelt- und Unternehmensanalyse gut strukturiert aufgebaut, eine gute Einleitung geschrieben und die einzelnen Fakten auf die verschiedenen Faktoren bezogen, auch die zu hohen Produktionskosten hast du raus gearbeitet. Die Aufgabe hast du sehr gut gemeistert. Ich hätte ein paar Verbesserungsvorschläge. Du hättest in der Aufgabe zunächst erklären können, was eine Umwelt- und Unternehmensanalyse überhaupt ist. Außerdem würde ich das Wort "Firma" durch Firma oder ähnliches ersetzen. Die Aufgabe zwei hast du gut beantwortet, du hättest eventuellentuell noch kurz auf den Stakeholderansatz eingehen können, der zum Verständnis der Aufgabenstellung notwendig war. Du hättest vielleicht noch den Schwerpunkt der Unternehmensanalyse ändern können, aber das ist nur ein kleiner Punkt. Allgemein könnte man


In [16]:
training_args = TrainingArguments(
    output_dir = 'gerpt2l-20',
    overwrite_output_dir = True,
    num_train_epochs = 10,
    per_device_train_batch_size = 16,
    per_device_eval_batch_size = 32,
    eval_steps = 400,
    save_steps = 800,
    warmup_steps = 500,
)
gerpt2l_trainer = Trainer(
    model = gerpt2l_model,
    args = training_args,
    data_collator = data_collator,
    train_dataset = train_dataset,
    eval_dataset = test_dataset,
    # compute_metrics = compute_metrics
    # prediction_loss_only = True,
)

/opt/conda/lib/python3.9/site-packages/torch/cuda/__init__.py:145: UserWarning: 
NVIDIA A100-SXM4-40GB with CUDA capability sm_80 is not compatible with the current PyTorch installation.
The current PyTorch install supports CUDA capabilities sm_37 sm_50 sm_60 sm_70.
If you want to use the NVIDIA A100-SXM4-40GB GPU with PyTorch, please check the instructions at https://pytorch.org/get-started/locally/

  warnings.warn(incompatible_device_warn.format(device_name, capability, " ".join(arch_list), device_name))


In [ ]:
gerpt2l_trainer.train()

/opt/conda/lib/python3.9/site-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 20629
  Num Epochs = 10
  Instantaneous batch size per device = 16
  Total train batch size (w. parallel, distributed & accumulation) = 32
  Gradient Accumulation steps = 1
  Total optimization steps = 6450
  Number of trainable parameters = 774030080


### Evaluation

In [ ]:
pip install evaluate

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 69 kB 3.1 MB/s 
     |████████████████████████████████| 432 kB 17.1 MB/s 
     |████████████████████████████████| 115 kB 53.5 MB/s 
     |████████████████████████████████| 212 kB 67.5 MB/s 
     |████████████████████████████████| 127 kB 43.8 MB/s 
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3


In [ ]:
from evaluate import load
perplexity = load("perplexity", module_type="metric")

In [ ]:
import pandas as pd
val = pd.read_table(dir + 'data/val_reviews.txt', header=None, names=['review'])

In [ ]:
gpt2_result = perplexity.compute(predictions=val.review, model_id='gpt2')
gpt2_result['mean_perplexity']

Downloading:   0%|          | 0.00/665 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/548M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/456k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

Using pad_token, but it is not set yet.


  0%|          | 0/87 [00:00<?, ?it/s]

333.75177682263757

In [ ]:
gpt2_result = perplexity.compute(predictions=val.review, model_id='gpt2')
gpt2_result['mean_perplexity']

Using pad_token, but it is not set yet.


  0%|          | 0/87 [00:00<?, ?it/s]

333.75177682263757

In [ ]:
german_gpt2_model.config

GPT2Config {
  "_name_or_path": "/content/drive/MyDrive/german-gpt2",
  "activation_function": "gelu_new",
  "architectures": [
    "GPT2LMHeadModel"
  ],
  "attn_pdrop": 0.0,
  "bos_token_id": 50256,
  "embd_pdrop": 0.0,
  "eos_token_id": 50256,
  "gradient_checkpointing": false,
  "initializer_range": 0.02,
  "layer_norm_epsilon": 1e-05,
  "model_type": "gpt2",
  "n_ctx": 1024,
  "n_embd": 768,
  "n_head": 12,
  "n_inner": null,
  "n_layer": 12,
  "n_positions": 1024,
  "reorder_and_upcast_attn": false,
  "resid_pdrop": 0.0,
  "scale_attn_by_inverse_layer_idx": false,
  "scale_attn_weights": true,
  "summary_activation": null,
  "summary_first_dropout": 0.1,
  "summary_proj_to_labels": true,
  "summary_type": "cls_index",
  "summary_use_proj": true,
  "task_specific_params": {
    "text-generation": {
      "do_sample": true,
      "max_length": 50
    }
  },
  "torch_dtype": "float32",
  "transformers_version": "4.23.1",
  "use_cache": true,
  "vocab_size": 50265
}

In [ ]:
model_name = 'german-gpt2'
german_gpt2_result = perplexity.compute(predictions=val.review, model_id=dir+model_name)
german_gpt2_result['mean_perplexity']

Using pad_token, but it is not set yet.


  0%|          | 0/87 [00:00<?, ?it/s]

IndexError: ignored

In [ ]:
model_name = 'bart-german'
bart_german_result = perplexity.compute(predictions=val.review, model_id=dir+model_name)
bart_german_result['mean_perplexity']

Using pad_token, but it is not set yet.


  0%|          | 0/87 [00:00<?, ?it/s]

IndexError: ignored

In [ ]:
model_name = 'german-gpt2'
model_path = dir + model_name
german_gpt2_model = AutoModelWithLMHead.from_pretrained(model_path)
gpt2_pipe = pipeline("text-generation", model=model_path)
gpt2_pipe("Zunächst kann ich sagen, dass deine erste Aufgabe gut gelöst wurde.", max_length=150)[0]["generated_text"]

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


'Zunächst kann ich sagen, dass deine erste Aufgabe gut gelöst wurde. Hier finde ich vor allem den Aufbau sehr gut. Deine Modularisierung ist ebenfalls gut gelungen. Jedoch fehlen die Module, die in einem Modul zusammengefasst werden sollen, siehe Diskurs 21/22. Hier bist du leider in der 2. Verzweigung und nicht richtig drüber gekommen. Hier kannst du besser darauf eingehen, welche Module du jetzt genauer erläuterst. Du kannst auch noch einmal die Abfolgen der Lanes genau überprüfen. Du hast hier zum Beispiel in der poolwidder aufgelistet was genau ausgelöscht wird. Ich hoffe mein Feedback war für dich hilfreich und ich sollte jetzt nicht mehr viel daran ändern.\n\nHey! :)Deine Idee finde wirklich super ist aber folgender Punkt: Was wäre'

In [ ]:
model_name = 'bart-german'
model_path = dir + model_name
bart_german_model = AutoModelWithLMHead.from_pretrained(model_path)
bart_german_model = pipeline("text-generation", model=model_path)
bart_german_model("Zunächst kann ich sagen, dass deine erste Aufgabe gut gelöst wurde.", max_length=150)[0]["generated_text"]

/usr/local/lib/python3.7/dist-packages/transformers/models/auto/modeling_auto.py:1066: FutureWarning: The class `AutoModelWithLMHead` is deprecated and will be removed in a future version. Please use `AutoModelForCausalLM` for causal language models, `AutoModelForMaskedLM` for masked language models and `AutoModelForSeq2SeqLM` for encoder-decoder models.
  FutureWarning,
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


'Zunächst kann ich sagen, dass deine erste Aufgabe gut gelöst wurde. Du kannst nur sagen, dass sie richtig, Pflicht- und Unterpunkt richtig haben.Du hast die Aufgabe 1 in der zweiten Aufgabe echt mühevoll gelöst. Das ärgert mich als :)Sonst war das super.LG\n\nZivilrecht: Du hast die Lösung sehr ausführlich und auch mit den erforderlichen Punkten erklärt. Die Lösung ist sehr genau und erkennt das Fachwissen aus dem juristischen und auch das Rechtsempfinden in einem simpler beim Sprachlich wäre eine Herausforderung. Deine Antwort war genau dieselbe wie die Aufgabe 1. Ih habe ich gar nicht nachgedacht, Sie soll genau dieselbe Antwort und haben den Paragraph in Tat und Worte sehr angenehm formuliert: es enthält nur Paragraphen 119,2'